# Defining Processes 

Processes convert a set of resources into another. The balance of this conversion is provided by the user (see section Conversion under the Defining Resources Tutorial). A few examples are list: 


## General Setup 

Consider the processes, Wind Farm (wf) and Proton Exchange Membrane Electrolysis (pem)
These require the resource wind, power, water (h2o), hydrogen (h2), oxygen (o2)

wind and h2o can be consumed, there is a demand for hydrogen and oxygen can be released

In [13]:
from energia import Currency, Process, Resource, Model, Periods

m = Model()
m.usd = Currency()
m.q = Periods()
m.y = 4 * m.q
m.declare(Resource, ['wind', 'power', 'h2o', 'h2', 'o2'])
m.declare(Process, ['wf', 'pem'])


m.wind.consume == True
m.h2o.consume == True

m.h2.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]
m.o2.release == True

--- General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
    Completed in 8.559226989746094e-05 seconds
--- General Resource Balance for h2o in (l, y): initializing constraint, adding consume(h2o, l, y)
    Completed in 5.650520324707031e-05 seconds
--- General Resource Balance for h2 in (l, q): initializing constraint, adding release(h2, l, q)
    Completed in 7.867813110351562e-05 seconds
--- Binding release in domain (h2, l, q)
    Completed in 0.00010943412780761719 seconds
--- General Resource Balance for o2 in (l, y): initializing constraint, adding release(o2, l, y)
    Completed in 6.747245788574219e-05 seconds


## With Positive Basis 

The basis is provided in the format Process(Resource | Conversion) For example, in the case of wind farm the basis is provided as power produced. Thus all the values for usd.spend need to be provided on the basis of power production 

In [14]:
m.wf(m.power) == -m.wind
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49

--- Binding operate in domain (wf, l, q)
--- Aspect (capacity) not defined, a variable will be created at l assuming y as the temporal index
    Completed in 0.0002970695495605469 seconds
--- Mapping operate across time from (wf, l, q) to (wf, l, y)
--- Creating map to (wf, l, y). Mapping operate: from (wf, l, q) to (wf, l, y)
    Completed in 6.389617919921875e-05 seconds


## With Negative Basis (or Conversion)

PEMs are typically sized based on their power consumption, thus all the values provide should be on the basis of power expended in the process

In [15]:
m.pem(-m.power) == 0.01248 * m.h2 + 0.09987 * m.o2 - 0.11235 * m.h2o
m.pem.capacity[m.usd.spend] == 1.55 * 10**6

## Locating Processes

Processes need to located explicitly 

If using multiple locations, the user can use m.location.locate(.. list of processes ..)

For single period models, it may be convenient to use m.locate(.. list of processes ..) which defaults to m.network.locate(..). Note that in a single location example, the location itself is the network

In [16]:
m.locate(m.pem, m.wf)

--- Assuming  pem capacity is unbounded in (l, y)
--- Assuming operation of pem is bound by capacity in (l, y)
--- Binding operate in domain (pem, l, y)
    Completed in 0.00011110305786132812 seconds
--- General Resource Balance for power in (l, q): initializing constraint, adding expend(power, l, q, operate, pem)
    Completed in 0.00010466575622558594 seconds
--- Mapping operate across time from (pem, l, q) to (pem, l, y)
--- Creating map to (pem, l, y). Mapping operate: from (pem, l, q) to (pem, l, y)
    Completed in 5.793571472167969e-05 seconds
--- General Resource Balance for h2 in (l, q): adding produce(h2, l, q, operate, pem)
    Completed in 8.559226989746094e-05 seconds
--- General Resource Balance for o2 in (l, y): adding produce(o2, l, y, operate, pem)
    Completed in 5.936622619628906e-05 seconds
--- General Resource Balance for h2o in (l, y): adding expend(h2o, l, y, operate, pem)
    Completed in 5.626678466796875e-05 seconds
--- Assuming  wf capacity is unbounded in 

The balances arising from production are only assumed once the process has beeen located. Upon location, the model can be optimized

In [17]:
m.usd.spend.opt()
m.capacity.sol()

--- Creating map to (usd, l, y). Mapping spend: from (usd, l, y, capacity, wf) to (usd, l, y)
    Completed in 9.918212890625e-05 seconds
--- Mapping spend: from (usd, l, y, operate, wf) to (usd, l, y)
    Completed in 5.1021575927734375e-05 seconds
--- Mapping spend: from (usd, l, y, capacity, pem) to (usd, l, y)
    Completed in 4.3392181396484375e-05 seconds
--- Generating m.mps
--- Creating gurobi model for m
Read MPS format model from file m.mps
Reading time = 0.00 seconds
M: 41 rows, 38 columns, 86 nonzeros
--- Optimizing m using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 41 rows, 38 columns and 86 nonzeros
Model fingerprint: 0xfd8097d4
Coefficient statistics:
  Matrix range     [1e-02, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Modeling Technology Choice

Let us continue with the example, but provide another option for power generation (solar PV)

In [18]:
m.solar = Resource()
m.solar.consume == True
m.pv = Process()

m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 30000
m.pv.capacity.x >= 0
m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 5670
m.pv.operate[m.usd.spend] == 90

m.locate(m.pv)

--- General Resource Balance for solar in (l, y): initializing constraint, adding consume(solar, l, y)
    Completed in 0.0002875328063964844 seconds
--- Binding capacity in domain (pv, l, y)
    Completed in 0.00010824203491210938 seconds
--- Binding capacity in domain (pv, l, y)
    Completed in 6.961822509765625e-05 seconds
--- Binding operate in domain (pv, l, q)
    Completed in 0.000164031982421875 seconds
--- Mapping operate across time from (pv, l, q) to (pv, l, y)
--- Creating map to (pv, l, y). Mapping operate: from (pv, l, q) to (pv, l, y)
    Completed in 5.626678466796875e-05 seconds
--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, pv)
    Completed in 9.632110595703125e-05 seconds
--- General Resource Balance for solar in (l, y): adding expend(solar, l, y, operate, pv)
    Completed in 0.00011372566223144531 seconds


## Semi-continuous Bounds 

These can be set using .x for any aspect. In the below example, the maximum capacity of wind farm (if set up) is 30000
The lower bound of 500 is only applied if wind farm is set up at all. Thus the values that wind farm can take are in the semi-continuous domain $\{0\} \cup [500, 30000]$


In [19]:
m.wf.capacity.x <= 30000
m.wf.capacity.x >= 500

--- Binding capacity in domain (wf, l, y)
    Completed in 0.0001785755157470703 seconds
--- Binding capacity in domain (wf, l, y)
    Completed in 0.00010704994201660156 seconds


In [20]:
m.usd.spend.opt()

--- Generating m.mps
--- Creating gurobi model for m
Read MPS format model from file m.mps
Reading time = 0.00 seconds
M: 57 rows, 54 columns, 125 nonzeros
--- Optimizing m using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 57 rows, 54 columns and 125 nonzeros
Model fingerprint: 0xebd4de5a
Variable types: 52 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 2e+02]
Presolve removed 47 rows and 40 columns
Presolve time: 0.00s
Presolved: 10 rows, 14 columns, 24 nonzeros
Variable types: 13 continuous, 1 integer (1 binary)
Found heuristic solution: objective 6.706731e+10

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 w

The solution for the two models can be compared

In [21]:
m.capacity.sol(compare=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note: binaries were only introduced in the second model

In [22]:
m.capacity.reporting.sol(compare=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>